In [31]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [32]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [33]:
print(data_dir)

c:\Users\hamin\Desktop\Supply-Chain-Analytic\data


# Extract

In [58]:
df_fulfillment = pd.read_csv(os.path.join(data_dir, 'fulfillment.csv'), index_col=None)
df_inventory = pd.read_csv(os.path.join(data_dir, 'inventory.csv'), index_col=None)
df_orders = pd.read_csv(os.path.join(data_dir, 'orders_and_shipments.csv'), index_col=None)

In [59]:
df_fulfillment.head()

,Product Name,Warehouse Order Fulfillment (days)
0,Perfect Fitness Perfect Rip Deck,8.3
1,Nike Men's Dri-FIT Victory Golf Polo,6.6
2,O'Brien Men's Neoprene Life Vest,5.5
3,Nike Men's Free 5.0+ Running Shoe,9.4
4,Under Armour Girls' Toddler Spine Surge Runni,6.3


# 2.Transform

## 2.1 Rename Columns

In [40]:
def rename_columns(df):
    # Convert from camel case to snake case
    df.columns = (
        df.columns
        .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
        .str.lower()
    )
    # Replace spaces with underscores
    df.columns = df.columns.str.replace(' ', '_')
    # Replace hyphens with underscores
    df.columns = df.columns.str.replace('-', '_')
    # Replace percentage signs with 'percent'
    df.columns = df.columns.str.replace('%', 'percent')
    # Remove parentheses
    df.columns = df.columns.str.replace('[()]', '', regex=True)
    # Replace multiple underscores with a single underscore
    df.columns = df.columns.str.replace('_+', '_', regex=True)
    # Remove leading and trailing underscores
    df.columns = df.columns.str.strip('_')
    
    return df

In [60]:
df_inventory = rename_columns(df_inventory)
df_fulfillment = rename_columns(df_fulfillment)
df_orders = rename_columns(df_orders)

In [64]:
df_fulfillment.columns

Index(['product_name', 'warehouse_order_fulfillment_days'], dtype='object')

## 2.2 Check Datatypes

In [65]:
df_fulfillment.dtypes

product_name                         object
warehouse_order_fulfillment_days    float64
dtype: object

In [45]:
df_inventory.dtypes

product_name                object
year_month                   int64
warehouse_inventory          int64
inventory_cost_per_unit    float64
dtype: object

In [46]:
df_orders.dtypes

order_id                    int64
order_item_id               int64
order_year_month            int64
order_year                  int64
order_month                 int64
order_day                   int64
order_time                 object
order_quantity              int64
product_department         object
product_category           object
product_name               object
customer_id                 int64
customer_market            object
customer_region            object
customer_country           object
warehouse_country          object
shipment_year               int64
shipment_month              int64
shipment_day                int64
shipment_mode              object
shipment_days_scheduled     int64
gross_sales                 int64
discount_percent           object
profit                      int64
dtype: object

# 3. Load

In [72]:
import pyodbc
from sqlalchemy import create_engine
import sqlalchemy

In [52]:
# Define the connection string using Windows Authentication
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=BOOK-3EOH2MPGSA\SQLEXPRESS;'
    'DATABASE=SUPPLY_CHAIN_ANALYTICS;'
    'Trusted_Connection=yes;'
)

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\hamin\AppData\Local\Temp\ipykernel_20484\3518523648.py:4: SyntaxWarning: invalid escape sequence '\S'
  'SERVER=BOOK-3EOH2MPGSA\SQLEXPRESS;'


In [81]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]

In [82]:
def load_table(df, table_name, schema_name):
    # Define the connection string using Windows Authentication
    conn_str = (
        'Driver={SQL Server};'
        'Server=BOOK-3EOH2MPGSA\\SQLEXPRESS;'
        'Database=SUPPLY_CHAIN_ANALYTICS;'
        'Trusted_Connection=yes;'
    )

    # Create a pyodbc connection
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Drop the table if it already exists
    cursor.execute(f"IF OBJECT_ID('{schema_name}.{table_name}', 'U') IS NOT NULL DROP TABLE {schema_name}.{table_name}")
    conn.commit()

    # Create the table schema
    columns = ', '.join([f'{col} {dtype}' for col, dtype in zip(df.columns, get_column_types(df))])
    create_table_sql = f"CREATE TABLE {schema_name}.{table_name} ({columns})"
    cursor.execute(create_table_sql)
    conn.commit()

    # Insert the data into the table
    insert_sql = f"INSERT INTO {schema_name}.{table_name} VALUES ({','.join(['?'] * len(df.columns))})"
    for _, row in df.iterrows():
        cursor.execute(insert_sql, *row.tolist())
    conn.commit()

    cursor.close()
    conn.close()
    print(f"DataFrame loaded successfully into {schema_name}.{table_name}")

In [79]:
df_fulfillment.dtypes

product_name                         object
warehouse_order_fulfillment_days    float64
dtype: object

In [83]:
load_table(df_fulfillment, 'fulfillment', 'original')

DataFrame loaded successfully into original.fulfillment


In [84]:
load_table(df_orders, 'orders', 'original')

DataFrame loaded successfully into original.orders


In [85]:
load_table(df_inventory, 'inventory', 'original')

DataFrame loaded successfully into original.inventory
